
# Running your own simulation!

### Step 1: Importing necessary libraries

In [1]:
import flowermd # wrapper we use on top of hoomd
import time #For measuring simulation time
import gsd # to work with simulation data files
import gsd.hoomd # to work with simulation data files directly coming from hoomd sims
import hoomd # MD simulation engine used for initializing and running sims
import mbuild as mb # library used to build Molecule objects
import numpy as np # common library used for a plethora of things, such as indexing, numeric constants, linear algebra, etc.
import warnings # used to provide the user warnings rather than errors
from flowermd.base import Pack, Simulation, System, Molecule # importing useful functions from flowermd to simplify initializing and running 
from flowermd.library import LJChain # lennard jones polymer chains with scalable length
from flowermd.library.forcefields import BeadSpring # forcefield template used in order to define interactions within sim
from flowermd.utils import get_target_box_number_density # used to define final density to shrink box to in order to minimize space
from mbuild.compound import Compound # base compound object to create molecule geometry
from mbuild.lattice import Lattice # lattice object to initialize lattice spacing and points
import unyt as u # module used for unit definitions
warnings.filterwarnings('ignore') # ignore warnings

/home/blakem/miniconda3/envs/mxene/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


### Step 2: Define flake geometry

In [2]:
class Flake(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) # define lattice vectors, points, and spacings for flakes
        Flakium = Compound(name="F", element="F") # defines an atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated flake lattice layers to the 'surface' compound, which represents our flake structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Flake, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]

### Step 3: Define forcefield, Weeks-Chandler-Anderson

In [3]:
ff = BeadSpring(
    r_cut=2**(1/6),  # r_cut value defines the radius in which a given particle will interact with another.
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains, epsilon = well depth, defines strength of attractive forces between two molecules
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes, sigma = distance between two particles where PE is zero
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # r0 = equilibrium distance of bonded particles, k = stiffness constant
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), # do not worry about dihedrals
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)

### Step 4: Define simulation parameters

In [4]:
N_chains = 50 # number of polymer chains
initial_dens = 0.001 # initial packing density to initialize system
final_dens = 0.3 # final packing density for shrinking
N_flakes = 5 # number of flakes
chain_length = 10 # length of polymer chains
dt = 0.005 # step size of simulation
temp = 1.0 # kT, temperature of simulation
write_frequency = 1000 # write frequency of simulation, the lower the more frames captured
shrink_steps = 5e5 # amount of steps to run shrink
sim_start = int(shrink_steps/write_frequency) # for use in analysis notebook, post-shrink simulation starting point
steps = 1e6 # amount of steps to run simulation for

### Step 5: Running on a CPU? Or a GPU?

In [5]:
device = hoomd.device.GPU() # change to hoomd.device.GPU() if you can/want to run on your GPU.Running

### Step 6: Initialize system

In [6]:
kg_chain = LJChain(lengths=chain_length,num_mols=N_chains) # initializing polymer chains
sheet = Flake(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False)) # initializing flakes
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=N_flakes), kg_chain], 
              density=initial_dens, packing_expand_factor = 6, seed=2) # packing chains and flakes into system
target_box = get_target_box_number_density(density=final_dens*u.Unit("nm**-3"),n_beads=(500+(N_chains*10))) # acquiring final density scaling with number density

### Step 7: Output files

In [7]:
gsd = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt_{temp}kT.gsd" # name of output gsd files
log = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt_{temp}kT.txt" # name of output log files
start_file = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt_{temp}kT_start.txt" # name of output start of sim

### Step 8: Initializing simulation, then running shrink -> simulation

In [8]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=device, dt = dt, 
                 gsd_write_freq=int(write_frequency), log_file_name = log, gsd_file_name = gsd) # initializing simulation
start_shrink = time.time()
sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=shrink_steps,tau_kt=100*sim.dt,period=10,thermalize_particles=True) # shrink simulation run
end_shrink = time.time()
start_run = time.time()
sim.run_NVT(n_steps=steps, kT=temp, tau_kt=dt*100) # simulation run
end_run = time.time()
sim.flush_writers() # updating data files
del sim # drop references so files are closed

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 500000; TPS: 779.81; ETA: 10.7 minutes
Step 2000 of 500000; TPS: 1306.8; ETA: 6.4 minutes
Step 3000 of 500000; TPS: 1701.77; ETA: 4.9 minutes
Step 4000 of 500000; TPS: 2047.37; ETA: 4.0 minutes
Step 5000 of 500000; TPS: 2287.96; ETA: 3.6 minutes
Step 6000 of 500000; TPS: 2485.09; ETA: 3.3 minutes
Step 7000 of 500000; TPS: 2687.32; ETA: 3.1 minutes
Step 8000 of 500000; TPS: 2718.43; ETA: 3.0 minutes
Step 9000 of 500000; TPS: 2837.38; ETA: 2.9 minutes
Step 10000 of 500000; TPS: 2939.06; ETA: 2.8 minutes
Step 11000 of 500000; TPS: 3033.78; ETA: 2.7 minutes
Step 12000 of 500000; TPS: 3026.98; ETA: 2.7 minutes
Step 13000 of 500000; TPS: 3125.52; ETA: 2.6 minutes
Step 14000 of 500000; TPS: 3195.24; ETA: 2.5 minutes
Step 15000 of 500000; TPS: 3153.27; ETA: 2.6 minutes
Step 16000 of 500000; TPS: 3232.34; ETA: 2.5 minutes
Step 17000 of 500000; TPS: 3304.03; ETA: 2.4 minutes
Step 18000 of 500000; TPS: 3369.64; ETA: 2.4 minutes
St

In [9]:
print(f"Shrink phase completed in {(end_shrink - start_shrink)} seconds")
print(f"Run phase completed in {(end_run - start_run)} seconds")
print(f"Total time: {((end_run - start_run)+(end_shrink - start_shrink))/60:.2f} minutes")

Shrink phase completed in 109.84944200515747 seconds
Run phase completed in 197.10290503501892 seconds
Total time: 5.12 minutes


In [10]:
with open(start_file, "w") as f:
    f.write(str(sim_start) + '\n')
    f.write(str(f"[BM]_{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt_{temp}kT") + '\n')
    f.write(str(N_chains) + '\n')
    f.write(str(chain_length) + '\n')
    f.write(str(N_flakes) + '\n')
    f.write(str(initial_dens) + '\n')
    f.write(str(final_dens) + '\n')
    f.write(str(dt) + '\n')
    f.write(str(temp) + '\n')
    f.write(str(steps) + '\n')
    f.write(str(steps/write_frequency) + '\n')
    f.write(str(end_shrink - start_shrink) + '\n')
    f.write(str(end_run - start_run) + '\n')
    f.write(str(((end_run - start_run) + (end_shrink - start_shrink))/60))

In [11]:
!mv "{log}" "{gsd}" "{start_file}" ../analysis/